## Play with loading in weights to a keras model and then exporting to a PyTorch model

In [1]:
import sys
# point path to genesis repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/genesis/analysis/splicing'
)

In [2]:
from definitions.generator.splirent_deconv_conv_generator_concat import load_generator_network
from genesis.generator import build_generator, st_sampled_softmax, st_hardmax_softmax
from pathlib import Path
from keras.models import load_model
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [3]:
model_name = 'genesis_splicing_cnn_target_isoform_00_pwm_and_multisample_hek_only_random_regions_50_epochs_harderentropy_generator.h5'
model_save_dir = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_pretrained_models/'

full_path = model_save_dir + model_name

In [4]:
generator_model = load_model(filepath=str(full_path), custom_objects={'K': K, 'st_sampled_softmax': st_sampled_softmax, 'st_hardmax_softmax': st_hardmax_softmax}, compile=False)

2024-03-10 15:35:05.000530: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-03-10 15:35:05.017520: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2024-03-10 15:35:05.019342: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557987c54380 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-10 15:35:05.019371: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Instructions for updating:
Use `tf.random.categorical` instead.



In [7]:
generator_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence_class_seed (InputLayer (32, 1)              0                                            
__________________________________________________________________________________________________
lambda_rand_sequence_class (Lam (32, 1)              0           sequence_class_seed[0][0]        
__________________________________________________________________________________________________
noise_input_1 (InputLayer)      (32, 100)            0                                            
__________________________________________________________________________________________________
noise_input_2 (InputLayer)      (32, 100)            0                                            
__________________________________________________________________________________________________
lambda_ran

In [16]:
for i, weight in enumerate(generator_model.get_weights()):
    print(i, weight.shape)

0 (101, 3456)
1 (3456,)
2 (7, 1, 256, 384)
3 (256,)
4 (256,)
5 (256,)
6 (256,)
7 (256,)
8 (8, 1, 192, 256)
9 (192,)
10 (192,)
11 (192,)
12 (192,)
13 (192,)
14 (7, 1, 128, 192)
15 (128,)
16 (128,)
17 (128,)
18 (128,)
19 (128,)
20 (8, 1, 128, 128)
21 (128,)
22 (128,)
23 (128,)
24 (128,)
25 (128,)
26 (8, 1, 128, 64)
27 (64,)
28 (64,)
29 (64,)
30 (64,)
31 (64,)
32 (8, 1, 64, 4)
33 (4,)
34 (1, 436)
35 (1, 436)


Get order of weights

In [10]:
len(generator_model.get_weights())

36

Got the weights!

Let's save the weights into a numpy object file

In [15]:
save_path = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_generator_weights/'
save_name = 'target_isoform_00.npy'

np.save(save_path+save_name, np.array(generator_model.get_weights(), dtype=object), allow_pickle=True)

Now we're gonna try loading in the weights into our PyTorch model

In [1]:
import sys
# point path to our repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps/src'
)

import numpy as np
from models.den import Generator
import torch
from data.old_dataset import create_sequence_templates
from torch import nn
from models.base_cnn import OracleCNN

/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps/project-env/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
save_path = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_generator_weights/'
save_name = 'target_isoform_00.npy'

target_isoform_net_weights = np.load(save_path+save_name, allow_pickle=True)

# list of numpy arrays
weights = target_isoform_net_weights.tolist()

In [3]:
device = torch.device('cuda')
# embedding_template, embedding_mask = create_sequence_templates()

# need to extract embedding template and mask from pretrained network
pretrained_embedding_template = torch.tensor(weights[-2]).to(device)
pretrained_embedding_mask = torch.tensor(weights[-1]).to(device)

n_samples = 10
n_classes = 1
seq_length = 109
latent_dim = 100
batch_size = 32

In [4]:
torch_generator = Generator(embedding_template=pretrained_embedding_template,
                                   embedding_mask=pretrained_embedding_mask,
                                   device=device,
                                   latent_dim=latent_dim,
                                   batch_size=batch_size,
                                   seq_length=seq_length,
                                   n_classes=n_classes,
                                   n_samples=n_samples)

In [5]:
torch_generator.generator_network.generator_network

ModuleList(
  (0): Linear(in_features=101, out_features=3456, bias=True)
  (1): ConvTranspose2d(384, 256, kernel_size=(7, 1), stride=(2, 1))
  (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ConvTranspose2d(256, 192, kernel_size=(8, 1), stride=(2, 1))
  (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ConvTranspose2d(192, 128, kernel_size=(7, 1), stride=(2, 1))
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Conv2d(128, 128, kernel_size=(8, 1), stride=(1, 1), padding=same)
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Conv2d(128, 64, kernel_size=(8, 1), stride=(1, 1), padding=same)
  (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Conv2d(64, 4, kernel_size=(8, 1), stride=(1, 1), padding=same)
)

In [5]:
running_keras_weight_idx = 0

for i, layer in enumerate(torch_generator.generator_network.generator_network):
    if isinstance(layer, nn.Linear):
        # transfer linear layer weights and biases
        assert tuple(layer.weight.data.shape) == weights[running_keras_weight_idx].T.shape
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx].T)
        running_keras_weight_idx += 1

        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.ConvTranspose2d):
        # transfer convtranspose2d weights and biases
        assert tuple(layer.weight.data.shape) == np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]).shape
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.Conv2d):
        # transfer conv2d weights and biases
        assert tuple(layer.weight.data.shape) == np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]).shape
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.BatchNorm2d):
        # transfer batch norm gamma, beta, running mean, running variance
        # order from keras model should be gamma, beta, moving mean, moving variance
        assert tuple(layer.weight.data.shape) == weights[running_keras_weight_idx].shape # this might need a transpose
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.running_mean.data.shape == weights[running_keras_weight_idx].shape
        layer.running_mean.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.running_var.data.shape == weights[running_keras_weight_idx].shape
        layer.running_var.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1

Now that we've loaded the model, let's test it

In [6]:
torch_generator.to(device)

Generator(
  (generator_network): GeneratorNetwork(
    (generator_network): ModuleList(
      (0): Linear(in_features=101, out_features=3456, bias=True)
      (1): ConvTranspose2d(384, 256, kernel_size=(7, 1), stride=(2, 1), padding=('v', 'a', 'l', 'i', 'd'))
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (3): ConvTranspose2d(256, 192, kernel_size=(8, 1), stride=(2, 1), padding=('v', 'a', 'l', 'i', 'd'))
      (4): BatchNorm2d(192, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (5): ConvTranspose2d(192, 128, kernel_size=(7, 1), stride=(2, 1), padding=('v', 'a', 'l', 'i', 'd'))
      (6): BatchNorm2d(128, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (7): Conv2d(128, 128, kernel_size=(8, 1), stride=(1, 1), padding=same)
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (9): Conv2d(128, 64, kernel_size=(8, 1), stride=(1, 1), padding=same)


In [7]:
torch_generator.eval()
samples = None
with torch.no_grad():
    samples = torch_generator()

TypeError: conv_transpose2d(): argument 'padding' (position 5) must be tuple of ints, not tuple

In [ ]:
samples[0][0][0].reshape((109, 4))

tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0.,

In [ ]:
samples[0].shape

In [10]:
# save generated samples
save_path = '/gpfs/commons/groups/knowles_lab/ting/'
save_name = 'pytorch_generated_sequences_target_isoform_00.npy'

np.save(save_path+save_name, samples[0].detach().cpu().numpy(), allow_pickle=True)

In [11]:
# also save pwm
save_path = '/gpfs/commons/groups/knowles_lab/ting/'
save_name = 'pytorch_optimized_pwm_target_isoform_00.npy'

np.save(save_path+save_name, samples[2].detach().cpu().numpy(), allow_pickle=True)

Double check length of data

In [ ]:
PATH_TO_DIRECTORY = '/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps'
dataset_path = PATH_TO_DIRECTORY + '/old_data/5SS_compressed.txt'
seq_len = 101
n = 265137
inputs = np.zeros((n, seq_len, 4))
prob_s1 = np.zeros(n)

with open(dataset_path) as f:
    ind = 0
    for line in f:
        mod_line = line.split('\t')
        print(mod_line)